<a href="https://colab.research.google.com/github/ArunK-ML/Project---Nutrition-Paradox-A-Global-View-on-Obesity-and-Malnutrition/blob/main/Nutrition_Paradox1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **⚖️ Nutrition Paradox: A Global View on Obesity and Malnutrition**

In [2]:
pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.7 MB/s eta 0:00:00


In [8]:
import requests
import pandas as pd
import pycountry

# -----------------------------
# Step 1: Fetch Data from APIs
# -----------------------------
url1 = "https://ghoapi.azureedge.net/api/NCD_BMI_30C"
url2 = "https://ghoapi.azureedge.net/api/NCD_BMI_PLUS2C"
url3 = "https://ghoapi.azureedge.net/api/NCD_BMI_18C"
url4 = "https://ghoapi.azureedge.net/api/NCD_BMI_MINUS2C"

response1 = requests.get(url1)
response2 = requests.get(url2)
response3 = requests.get(url3)
response4 = requests.get(url4)

Obesity_adults = response1.json()
Obesity_children = response2.json()
Malnutrition_adults = response3.json()
Malnutrition_children = response4.json()

# -----------------------------
# Step 2: Convert JSON to DataFrames
# -----------------------------
Obesity_adults_df = pd.DataFrame(Obesity_adults["value"])
Obesity_children_df = pd.DataFrame(Obesity_children["value"])
Malnutrition_adults_df = pd.DataFrame(Malnutrition_adults["value"])
Malnutrition_children_df = pd.DataFrame(Malnutrition_children["value"])

# Add "age_group" column
Obesity_adults_df["age_group"] = "Adult"
Obesity_children_df["age_group"] = "Child"
Malnutrition_adults_df["age_group"] = "Adult"
Malnutrition_children_df["age_group"] = "Child"

# -----------------------------
# Step 3: Combine datasets
# -----------------------------
df_obesity = pd.concat([Obesity_adults_df, Obesity_children_df], ignore_index=True)
df_malnutrition = pd.concat([Malnutrition_adults_df, Malnutrition_children_df], ignore_index=True)

# -----------------------------
# Step 4: Select and Rename Columns
# -----------------------------
columns_map = {
    "ParentLocationCode": "Region",
    "Dim1": "Gender",
    "TimeDim": "Year",
    "Low": "LowerBound",
    "High": "UpperBound",
    "NumericValue": "Mean_Estimate",
    "SpatialDim": "Country"
}

selected_cols = list(columns_map.keys()) + ["age_group"]

df_obesity_subset = df_obesity[selected_cols].rename(columns=columns_map)
df_malnutrition_subset = df_malnutrition[selected_cols].rename(columns=columns_map)

# -----------------------------
# Step 5: Data Type Conversion
# -----------------------------
df_obesity_subset["Gender"] = df_obesity_subset["Gender"].astype("category")
df_obesity_subset["age_group"] = df_obesity_subset["age_group"].astype("category")
df_obesity_subset["Year"] = df_obesity_subset["Year"].astype(int)
df_malnutrition_subset["Year"] = df_malnutrition_subset["Year"].astype(int)

# -----------------------------
# Step 6: Filter Year 2012–2022
# -----------------------------
df_obesity_subset_filter = df_obesity_subset[
    (df_obesity_subset["Year"] >= 2012) & (df_obesity_subset["Year"] <= 2022)
].reset_index(drop=True)

df_malnutrition_subset_filter = df_malnutrition_subset[
    (df_malnutrition_subset["Year"] >= 2012) & (df_malnutrition_subset["Year"] <= 2022)
].reset_index(drop=True)

# -----------------------------
# Step 7: Replace Gender Codes
# -----------------------------
gender_map = {
    "SEX_FMLE": "Female",
    "SEX_BTSX": "Both",
    "SEX_MLE": "Male"
}

df_obesity_subset_filter.loc[:, "Gender"] = df_obesity_subset_filter["Gender"].replace(gender_map)
df_malnutrition_subset_filter.loc[:, "Gender"] = df_malnutrition_subset_filter["Gender"].replace(gender_map)

# -----------------------------
# Step 8: Replace Country Codes with Country Names (in-place)
# -----------------------------
def code_to_country(code):
    try:
        return pycountry.countries.get(alpha_2=code.upper()).name
    except:
        return "Unknown"

df_obesity_subset_filter["Country"] = df_obesity_subset_filter["Country"].apply(code_to_country)
df_malnutrition_subset_filter["Country"] = df_malnutrition_subset_filter["Country"].apply(code_to_country)

# -----------------------------
# Step 9: Output Preview
# -----------------------------
print("✅ Obesity Data (2012–2022):")
print(df_obesity_subset_filter.head())

print("\n✅ Malnutrition Data (2012–2022):")
print(df_malnutrition_subset_filter.head())


✅ Obesity Data (2012–2022):
  Region  Gender  Year  LowerBound  UpperBound  Mean_Estimate  Country  \
0   SEAR    Male  2013    3.004463    4.399506       3.682037  Unknown   
1    EMR    Male  2020   15.793777   19.245727      17.487641  Unknown   
2    EMR    Male  2020   23.089589   33.815336      28.335530  Unknown   
3    EUR  Female  2014   24.924766   31.213260      27.948704  Unknown   
4    EUR  Female  2019   28.160682   35.574925      31.893221  Unknown   

  age_group  
0     Adult  
1     Adult  
2     Adult  
3     Adult  
4     Adult  

✅ Malnutrition Data (2012–2022):
  Region  Gender  Year  LowerBound  UpperBound  Mean_Estimate  Country  \
0    AFR  Female  2021    3.972036    8.077021       5.795110  Unknown   
1    WPR    Both  2017    2.428230    3.842155       3.090652  Unknown   
2    AFR  Female  2016    6.655668   10.253121       8.371724  Unknown   
3   None    Both  2021   12.242323   14.442866      13.336353  Unknown   
4    EMR    Male  2014    1.253843    3

/tmp/ipython-input-8-99418057.py:89: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_obesity_subset_filter.loc[:, "Gender"] = df_obesity_subset_filter["Gender"].replace(gender_map)
/tmp/ipython-input-8-99418057.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Male', 'Male', 'Male', 'Female', 'Female', ..., 'Both', 'Female', 'Male', 'Male', 'Male']
Length: 27720
Categories (3, object): ['Both', 'Female', 'Male']' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df_obesity_subset_filter.loc[:, "Gender"] = df_obesity_subset_filter["Gender"].replace(gender_map)


In [6]:
df_obesity_subset_filter.head()

,Region,Gender,Year,LowerBound,UpperBound,Mean_Estimate,Country,age_group,Region_Name,Country_Name
0,SEAR,Male,2013,3.004463,4.399506,3.682037,LKA,Adult,South-East Asia Region,Unknown
1,EMR,Male,2020,15.793777,19.245727,17.487641,IRN,Adult,Eastern Mediterranean Region,Unknown
2,EMR,Male,2020,23.089589,33.815336,28.335530,IRQ,Adult,Eastern Mediterranean Region,Unknown
3,EUR,Female,2014,24.924766,31.213260,27.948704,GRL,Adult,Europe,Unknown
4,EUR,Female,2019,28.160682,35.574925,31.893221,ARM,Adult,Europe,Unknown
